In [1]:
# Import libraries
import pandas as pd
from geopy import distance
pd.set_option('display.max_columns', None)
import numpy as np

In [53]:
# Read in entrie data set with lat and long
data = pd.read_csv("../pool data/ltrm_water_data_lat_long.csv")


/Users/alainastockdill/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,12,14,17,19,21,23,25,32,48,51,52,55,57,59,61,63,65,67,70,71,72,73,74,75,76,77,126,129) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [54]:
# Filter out the data that we want
data = data[['SHEETBAR','TN','TP','TEMP','DO','TURB','COND','VEL','SS','WDP','CHLcal','SECCHI', 'LONGITUDE', 'LATITUDE', 'DATE']]

In [55]:
# Determine which columns are duplicates and count how many duplicates there are
duplicates = data.groupby(['SHEETBAR']).size()

In [56]:
# reset index of duplicates and rename the column
duplicates = pd.DataFrame(duplicates, columns = ['count']).reset_index()
duplicates = duplicates[duplicates['count'] > 1]

In [42]:
# Create empty datafram
collapsed_data = pd.DataFrame()

In [58]:
data_dups = data[pd.Series(data["SHEETBAR"]).isin(duplicates["SHEETBAR"])]
collapsed_data = data[-pd.Series(data["SHEETBAR"]).isin(duplicates["SHEETBAR"])]

In [37]:
# Loops through the duplicates data to find the 
for i in range(0, duplicates.size):
  collapsed_data.append(data.loc[data['SHEETBAR'] == duplicates.at[i,"SHEETBAR"]])

  # Would then append the columns to the blank dataframe

KeyError: 156474

In [39]:
# Further sorts the duplicates data to ensure no single values are in the list


In [ ]:
# For each SHEETBAR in one samp, add immediately into the collapsed data

In [ ]:
# For each SHEETBAR in duplicate, create a temp dataframe with duplicate rows 